# Dominik Adamczyk
## Laboratorium 4 - rozwiązania 

In [598]:
import numpy as np
import string
from collections import Counter
from itertools import combinations, combinations_with_replacement
import sklearn.cluster

### N-grams

W zadaniu będę posługiwał się n-gramami dla metryk cosinusowej, Sorensena-Dice'a, euklidesowej.

In [599]:
def generate_ngrams(words, n):
    ngrams = [words[i:i+n] for i in range(len(words) - n + 1)]
    return Counter(ngrams)

## Metryki

1. Zaimplementuj przynajmniej 3 "metryki" spośród wymienionych: cosinusowa, LCS, DICE, euklidesowa, Levenshteina.

Wszystkie implementowane metryki będą w pewien sposób normalizowane, tak żeby teksty takie same po porównaniu metryką dawały wynik 0 (tożsame z odległością dwóch punktów na płaszczyźnie, które są w tym samym miejscu), a teksty całkowicie różne dawały w wyniku 1. Dzięki temu metryki będą w pewien sposób porównywalne do siebie.

### "Metryka" cosinusowa

$$
\begin{equation}
    \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}= \frac{\sum\limits_{i=1}^{n} A_i B_i}{\sqrt{\sum\limits_{i=1}^{n} A_i^2} \sqrt{\sum\limits_{i=1}^{n} B_i^2}}
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &\mathbf{A}\text{ and }\mathbf{B} \text{ są porównywanymi wektorami}\\
    &n \text{ jest wymiarem wektora}\\
    &\theta \text{ reprezentuje kąt pomiędzy } \mathbf{A} \text{ i } \mathbf{B} \text{ wielowymiarowej przestrzeni}
    \end{aligned}
\end{equation}
$$

Wynik należy do przedziału $[-1,1]$. W tym zadaniu wynik będę odejmował od liczby 1 tak, aby takie same wektory miały odległość od siebie równą 0.


In [600]:
def cosine_similarity(A, B, n=2):
    n = min(len(A), len(B), n)
    A_grams = generate_ngrams(A, n)
    B_grams = generate_ngrams(B, n)
    if n == 0 : return 1
    return 1 - round(sum(A_grams[key] * B_grams[key] for key in A_grams.keys() & B_grams.keys()) / \
            (np.linalg.norm(list(A_grams.values())) * np.linalg.norm(list(B_grams.values()))), 15)


### Dystans euklidesowy 

$$
\begin{equation}
    d(x,y) = \sqrt{\sum_{i=1}^{n}(x_i-y_i)^2}
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &d(x,y) \text{ jest dystansem euklidesowym} \\
    &x_i, y_i \text{ są wartościami wektorów w i-tym wymiarze } x \text{ and } y \\
    &n \text{ jest wymiarem wektora}
    \end{aligned}
\end{equation}
$$

In [601]:
def euclidean_distance(A, B, n=2):
    n = min(len(A), len(B), n)
    A_grams = generate_ngrams(A, n)
    B_grams = generate_ngrams(B, n)
    if n == 0 : return 1
    keys = set(A_grams.keys()) | set(B_grams.keys())
    A_vals = np.array([A_grams[key] for key in keys])
    B_vals = np.array([B_grams[key] for key in keys])
    return np.linalg.norm(A_vals - B_vals) / np.linalg.norm(np.concatenate((A_vals, B_vals)))

### Współczynnik Dice

$$
\begin{equation}
    \text{Dice}(A, B) = \frac{2 |A \cap B|}{|A| + |B|} 
    \qquad\begin{aligned}
    &\text{gdzie:} \\
    &A \text{ and } B \text{ reprezentują dwa porównywane zbiory} \\
    &|A| \text{ i } |B| \text{ reprezentuje liczbę elementów zbioru} \\
    &\text{i } |A \cap B| \text{ reprezentuje liczbę elementów przecięcia zbiorów}
    \end{aligned}
\end{equation}
$$

In [602]:
def dice(A, B, n=2):
    n = min(len(A), len(B), n)
    if n==0: return 1
    A = set(generate_ngrams(A, n).keys())
    B = set(generate_ngrams(B, n).keys())
    
    return 1 - 2 * len(A & B) / (len(A) + len(B))
    

### Odległość Levenshteina

In [603]:
def levenshtein_distance(str1, str2):
    m = len(str1)
    n = len(str2)
    if n == 0 or m == 0: return 1
    dp = [[0 for j in range(n+1)] for i in range(m+1)]

    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[-1][-1] / max(m, n)

### LCS - Najdłuższy wspólny podciąg (Longest Common Subsequence)

In [604]:
def lcs(str1, str2):
    m = len(str1)
    n = len(str2)
    if n == 0 or m == 0: return 1
    lcs_table = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i-1] == str2[j-1]:
                lcs_table[i][j] = lcs_table[i-1][j-1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i][j-1], lcs_table[i-1][j])

    return 1 - lcs_table[-1][-1] / max(m, n)

### Czy to w ogóle działa?

Poniżej sprawdzę czy metryki dają (w przybliżeniu) oczekiwane wyniki z przedziału $[0, 1]$. 

1. Teksty takie same

In [605]:
t1 = "Ala Ma kotA"
t2 = t1

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.0
Odległość euklidesowa:  0.0
Współczynnik Dice:  0.0
Odległość levenshteina:  0.0
Najdłuższy wspólny podciąg:  0.0


2. Teksty zupełnie różne

In [606]:
t1 = "Ala Ma kotA"
t2 = "PieS_Nie_zJe"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  1.0
Odległość euklidesowa:  1.0
Współczynnik Dice:  1.0
Odległość levenshteina:  1.0
Najdłuższy wspólny podciąg:  1.0


3. Teksty bardzo podobne

In [607]:
t1 = "Ala ma mopsa"
t2 = "Ala ma psa"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.14365116142232504
Odległość euklidesowa:  0.3922322702763681
Współczynnik Dice:  0.17647058823529416
Odległość levenshteina:  0.16666666666666666
Najdłuższy wspólny podciąg:  0.16666666666666663


3. Teksty lekko podobne

In [608]:
t1 = "Ala ma mopsa"
t2 = "Owce lubią mopy"

print("Metryka cosinusowa: ", cosine_similarity(t1, t2))
print("Odległość euklidesowa: ", euclidean_distance(t1, t2))
print("Współczynnik Dice: ", dice(t1, t2))
print("Odległość levenshteina: ", levenshtein_distance(t1, t2))
print("Najdłuższy wspólny podciąg: ", lcs(t1, t2))


Metryka cosinusowa:  0.723973776263058
Odległość euklidesowa:  0.8509629433967631
Współczynnik Dice:  0.7391304347826086
Odległość levenshteina:  0.7333333333333333
Najdłuższy wspólny podciąg:  0.6666666666666667


## Ocena jakości klasteryzacji

2. Zaimplementuj przynajmniej 1 sposób oceny jakości klasteryzacji (np. indeks Daviesa-Bouldina)


W mojej implementacji centroidem będzie tekst w klastrze który ma najmniejszą odległość do wszystkich pozostałych tekstów według zadanej miary.
Decyzja spowodowana jest dopasowaniem algorytmu do wszystkich zaimplementowanych miar - w dwóch przypadkach tekst nie jest zamieniany na wektor częstości (dla LCS i Levenshteina)

### Indeks Daviesa-Bouldina

In [609]:
def get_centroid(cluster, metric):
    k = len(cluster)
    dist = np.zeros(k)
    for i, j in combinations(range(k), 2):
        to_add = metric(cluster[i], cluster[j])
        dist[i] += to_add
        dist[j] += to_add
    cluster = zip(dist, cluster)
    return min(cluster, key=lambda x : x[0])[1]
    

def intra_cluster_dispersion(cluster, metric):
    centroid = get_centroid(cluster, metric)
    return (sum([metric(text, centroid) ** 2 for text in cluster]) / len(cluster)) ** 0.5

def davies_bouldin_index(clusters, metric):
    n = len(clusters)
    centroids = [get_centroid(cluster, metric) for cluster in clusters]
    Si = [intra_cluster_dispersion(cluster, metric) for cluster in clusters]
    
    D_sum = 0
    
    for i in range(n):
        tmp = 0
        for j in range(n):
            if i == j: continue
            tmp = max(tmp, (Si[i] + Si[j]) / metric(centroids[i], centroids[j]))
        D_sum += tmp
    
    return D_sum / n

### Indeks Dunna

Definiuję wielkość klastra jako odległość między dwoma najodleglejszymi punktami w tym klastrze. Możliwości definicji jest więcej np. odległość wszystkich punktów danego klastra od ich centroidu.

Dystans pomiędzy klastrami zdefiniuję jako odległość między dwoma najbliższymi punktami należącymi do różnych klastrów.

Powyższe definicje są oryginalnymi dystansami zaproponowanymi przez Dunna.

In [610]:
def cluster_size(cluster, metric):
    return max(metric(ti, tj) for ti, tj in combinations_with_replacement(cluster, 2))

def intercluster_distance(cluster1, cluster2, metric):
    return min(metric(ti, tj) for ti in cluster1 for tj in cluster2)
    
def dunn_index(clusters, metric):
    numerator = min(intercluster_distance(c1, c2, metric) for (c1, c2) in combinations(clusters, 2))
    denominator = max(cluster_size(cluster, metric) for cluster in clusters)
    return numerator / denominator


### Czy to w ogóle działa?

Poniżej sprawdzę czy zaimplementowane przezemnie metody klasteryzacji dają sensowne wyniki. Dla indeksu Daviesa im lepsza klasteryzacja tym mniejszy wynik funkcji oceniającej, w przeciwieństwie do indeksu Dunna gdzie im lepsza klasteryzacja tym większy wynik.

1. Słaba klasteryzacja - teksty nie są za bardzo podobne do siebie

In [611]:
c = [["alamakot", "piesjemleko", "spaniefajnejest"],
     ["alkotomat", "mieszanie", "ciastko waniliowe"],
     ["lubie szarlotke", "ale jabłka muszą być dobre"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  1.6588014978165446
Dunn index, lcs:  0.5555555555555555
Davies-Bouldin index, euclidean:  1.5393580236310427
Dunn index, euclidean:  0.6831300510639733


2. Lepsza klasteryzacja - teksty w klastrach są bardziej zbliżone do siebie

In [612]:
c = [["alamakot", "ala ma kota", "ala nie ma kota"],
     ["ciasto wanilinowe", "ciastko tiramisu waniliowe", "ciastko waniliowe"],
     ["lubie szarlotke", "szarlotka szarlotka"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  0.8321434903110966
Dunn index, lcs:  1.1259259259259258
Davies-Bouldin index, euclidean:  0.899257071215224
Dunn index, euclidean:  1.1896993802573046


3. Słaba klasteryzacja - klastry są podobne do siebie

In [613]:
c = [["alamakot", "ala ma kota", "ala nie ma kota"],
     ["ala alkota ma", "ala kot", "kot ala"],
     ["alkomat", "ala z kotem"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  2.480104415122002
Dunn index, lcs:  0.40625
Davies-Bouldin index, euclidean:  1.769539479896063
Dunn index, euclidean:  0.4364357804719848


4. Lepsza klasteryzacja - klastry nie są podobne do siebie

In [614]:
c = [["alamakot", "trwonić"],
     ["xxxxz", "zzzzzzz", "ssssss"],
     ["cccccc", "dddddd"]]
print("Davies-Bouldin index, lcs: ", davies_bouldin_index(c, lcs))
print("Dunn index, lcs: ", dunn_index(c,lcs))

print("Davies-Bouldin index, euclidean: ", davies_bouldin_index(c, euclidean_distance))
print("Dunn index, euclidean: ", dunn_index(c,euclidean_distance))

Davies-Bouldin index, lcs:  1.438059209240504
Dunn index, lcs:  1.0
Davies-Bouldin index, euclidean:  1.5236033621142735
Dunn index, euclidean:  1.0


Wygląda na to, że metody oceniania klasteryzacji działają poprawnie, a przynajmniej dają oczekiwane względne wyniki

## Stoplista
3. Stwórz stoplistę najczęściej występujących słów i zastosuj ją jako pre-processing dla nazw. Algorytmy klasteryzacji powinny działać na dwóch wariantach: z pre-processingiem i bez pre-processingu.


Znajduję częstość słów w klasteryzowanym tekści i zwracam pewien procent najpopularniejszych słów

In [615]:
def stoplist(text, frequency): # frequency from interval [0, 1]
    words = [word for line in text for word in line.split(' ')]
    words = list(Counter(words).items())
    words.sort(key=lambda x:x[1])
    words = list(map(lambda x : x[0], words))
    return set(words[-int(frequency * len(words)):])


### Preprocessing tekstu

Tekst przed użyciem w metrykach jest konwertowany na małe litery, a także usuwane są znaki interpunkcyjne (o ile będziemy tego chciali użyć w programie, ja chyba będę chciał).


In [616]:
def preprocess_punctuation(text):
    out = []
    for l in text:
        lowercase_text = l.lower()
    
        out.append(lowercase_text.translate(str.maketrans('', '', string.punctuation)))
    
    return out


In [617]:
example_text = ["The quIck brown $fox j:umPs OV@er the lazy !doG. 123456789.com"]
preprocess_punctuation(example_text)

['the quick brown fox jumps over the lazy dog 123456789com']

In [618]:
def preprocess_stoplist(text, frequency):
    to_remove = stoplist(text, frequency)
    out = []
    for l in text:
        out.append(" ".join([w for w in l.split(" ") if w not in to_remove]))
    return out

In [619]:
def preprocess_all(text, frequency):
    text = preprocess_punctuation(text)
    return preprocess_stoplist(text, frequency)
    

### Wczytanie tekstu

In [620]:
with open("lines.txt", "r", encoding="UTF-8") as f:
    text = f.read().split("\n")

for el in text[:10]: print(el)

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA
''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611
''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939
"2TIGERS GROUP LIMITED"  ROOM 504 JINSHAZHOU SHANGSHUI ROAD,  GUANGZHOU 510160
"ALDETRANS" LLC, 105066, MOSCOW, RUSSIA, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89
"A-LIFT",JSC 1 PROSPEKT MARSHALA ZHUKOVA,MOSCOW 123308,RUSSIA  T: +7(495)784-7961
"ALISA" LTD, 1/5 Derbenevskaya str., Moscow, Russia Tel./Fax: (495) 987-13-07 postal code: 115114


### Test preprocessingu

In [621]:
print(stoplist(text, 0.01))

{'PETERSBURG,', '', 'SP.Z.O.O.', 'KOREA', 'OY', 'SA', 'WORLD', 'OCEAN', 'UL', 'A,', 'FLOOR', 'ST.PETERSBURG', 'IMPORT&EXPORT', 'O', 'PHONE:', 'SHIPPING', 'GLOBAL', '10', 'RUSSIA', 'WEST', 'INDUSTRY', 'HANGZHOU', '66', 'TO', 'SP.ZO.O.', '82', 'PANTOS', 'DISTRICT,', '58', 'AIR&SEA', 'ZHEJIANG', 'QINGDAO', '17', 'DISTRICT', 'PRODUCTS', 'HELSINKI', '+48', 'ST.PETERSBURG,', '(CHINA)', 'NO.', 'OOO', 'LINES', 'PARK', 'STR.,', 'RUSSIA,', '50', '22', 'O/B', '8', 'INTL', 'C/O', 'BUILDING', 'GDYNIA,', 'INN', 'COMPANY', 'GDYNIA,POLAND', 'Sp.', 'PANALPINA', 'TEL', 'THAILAND', 'CITY', 'NORTH', 'KONG', 'CO.,LTD.', 'ZONE,', '39', 'TOWER', 'BOX', 'SAVINO', 'EORI', 'BLDG', 'AS', 'GDYNIA', 'CENTER', 'LTD', 'FREIGHT', '4', 'LOGISTIC', 'Z.O.O.', 'BLUE', '83', 'NO', 'NEW', 'TAIWAN', 'GEODIS', 'TRANSPORT', 'C', ',', 'SOUTH', 'CARGO', 'IMPORT', '495', '70', 'A', '9', 'LIMITED', 'O.O.', 'BRANCH', '31', 'RD', 'PLAZA', 'SP.', 'ATTN:', 'TEL.', 'LIFLYANDSKAYA', 'LINE', 'KUEHNE', 'HONGKONG', '00', 'B', 'TOWER,', 'D

usuwanie punktuacji

In [622]:
for el in preprocess_punctuation(text)[:10]: print(el)

11692589 rd tuna canners ltd portion 1004 siar north coast road pobox 2113 madang papua new guinea
pa interior ltd bolshaya lubyanka street 164 moscow 101000 russia innkpp 7704550148770801001 4959848611
ssontex  spzooimportexportulprzeclawska 5 03879 warszawapoland nip 1130117669
ssontexspzooimportexport ulprzeclawska 5 03879 warszawapoland nip 1130117669 telfax0048022217 6532
topex sp z oo spolka komandytowa ul pograniczna 24  02285 warszawa poland
master plus coltd 143000russiamoodinsovo mojaiskoe shosse153g tel7495 7273939
2tigers group limited  room 504 jinshazhou shangshui road  guangzhou 510160
aldetrans llc 105066 moscow russia tokmakov lane 11 tel74956410389
aliftjsc 1 prospekt marshala zhukovamoscow 123308russia  t 74957847961
alisa ltd 15 derbenevskaya str moscow russia telfax 495 9871307 postal code 115114


usuwanie ze stoplisty

In [623]:
for el in preprocess_stoplist(text, 0.01)[:10]: print(el)

/11692589 TUNA CANNERS, PORTION 1004, SIAR COAST P.O.BOX 2113, MADANG, PAPUA GUINEA
''PA INTERIOR'' BOLSHAYA LUBYANKA 16/4 101000, INN/KPP 7704550148//770801001 495-984-8611
''SSONTEX'' Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
''TOPEX O.O.'' KOMANDYTOWA POGRANICZNA 2/4 02-285
'MASTER PLUS CO.,LTD.' 143000,RUSSIA,MO,ODINSOVO, MOJAISKOE, SHOSSE,153G TEL:+7495 7273939
"2TIGERS LIMITED" 504 JINSHAZHOU SHANGSHUI 510160
"ALDETRANS" LLC, 105066, TOKMAKOV LANE, 11. TEL:+7(495)641-03-89
"A-LIFT",JSC PROSPEKT MARSHALA ZHUKOVA,MOSCOW 123308,RUSSIA T: +7(495)784-7961
"ALISA" 1/5 Derbenevskaya str., Moscow, Russia Tel./Fax: (495) 987-13-07 postal code: 115114


In [624]:
for el in preprocess_all(text, 0.01)[:10]: print(el)

11692589 tuna canners portion 1004 siar coast 2113 madang papua guinea
pa interior bolshaya lubyanka 164 101000 innkpp 7704550148770801001 4959848611
ssontex spzooimportexportulprzeclawska 03879 warszawapoland nip 1130117669
ssontexspzooimportexport ulprzeclawska 03879 warszawapoland nip 1130117669 telfax0048022217 6532
topex komandytowa pograniczna 24 02285
master plus 143000russiamoodinsovo mojaiskoe shosse153g tel7495 7273939
2tigers 504 jinshazhou shangshui 510160
aldetrans 105066 tokmakov lane tel74956410389
aliftjsc prospekt marshala zhukovamoscow 123308russia t 74957847961
alisa derbenevskaya 9871307 postal code 115114


### Klasteryzacja

4. Wykonaj klasteryzację zawartości załączonego pliku (lines.txt) przy użyciu  metryk zaimplementowanych w pkt. 1. Każda linia to adres pocztowy firmy, różne sposoby zapisu tego samego adresu powinny się znaleźć w jednym klastrze.

In [625]:
def clusterize(text, metric, eps=0.9, preprocess=True, frequency=0.02):
    if preprocess:
        text = preprocess_all(text, frequency)
    dist = [[metric(la, lb) for la in text] for lb in text]
    clust = sklearn.cluster.DBSCAN(eps=eps, min_samples=1).fit(dist)
    return clust.labels_

def text_clusters(text, metric, eps=0.8, preprocess=True, frequency=0.02):
    labels = clusterize(text, metric, eps=0.8, preprocess=preprocess, frequency=0.02)
    clusters = [[] for _ in range(max(labels) + 1)]
    for idx, cl in enumerate(labels):
        clusters[cl].append(text[idx])
    return clusters

def clusters_from_labels(text, labels):
    clusters = [[] for _ in range(max(labels) + 1)]
    for idx, cl in enumerate(labels):
        clusters[cl].append(text[idx])
    return clusters

## Testowanie
5. Porównaj jakość wyników sposobami zaimplementowanymi w pkt. 2.


Na początek spróbuję zwizualizować czy klasteryzacja działa.

Jako że algorytmy tutaj używane mają złożoność $O(n^2)$ - gdzie n to długość linijki (np. lcs) i są powtarzane po $O(m^2)$ razy, to sklasteryzowanie wszystkich linijek z pliku lines.txt zajmuje dużo czasu. Dlatego będę korzystać z pierwszych 150 linii załączonego pliku.

In [626]:
text = text[:150]
test_labels = clusterize(text, dice)
print(test_labels)


[ 0  1  2  2  3  4  5  6  7  8  9 10 11 11 11 11 12 12 12  9  9 13 14 14
 15 16 17 18 18 19 20 21 22 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36
 37 37 38 39 39 39 40 40 40 40 41 41 41 41 41 41 42 43 43 44 44 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 58 59 60 61  2 62 63 64 65 66 65 66
 67 67 67 67 67 67 68  3  3  3  3  3  3  3  3 69  3  3 70 71 72 72 72 73
 63 74 75 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 90 91 92 93 93
 93 93 93 93 93 55]


In [627]:
test_clusters = text_clusters(text, dice)
for cluster in test_clusters[:30]:
    for line in cluster:
        print(line)
    print("\n" + "#"*10 + "\n")

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

##########

''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611

##########

''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
"SSONTEX" SP.ZO.O IMPORT-EXPORT 03-879 WARSZAWA UL PRZECLAWSKA 5 NIP:113-01-17-669

##########

''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4, 02-285 WARSZAWA.POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O."SP.K. UL,POGRANICZNA 2/4 02-285 WARSZAWA
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02--285 WARSZAWA
"TOPEX SP. Z O. O." SP. K. UL.POGRANICZNA 2/4,02-285 WARSZAWA,POLAND
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2

#### Klasteryzacja rzeczywiście działa przy wybranej metryce, chociaż zdarzają się pojedyncze błędy

In [628]:
def get_correct_clusters(text):
    with open('clusters.txt', 'r') as f:
        clusters = list(f)
    # print(clusters)
    correct_clusters = []
    for line in text:
        line += "\n"
        c = 0
        for cline in clusters:
            if cline[0] == "#":
                c += 1
            elif cline == line:
                correct_clusters.append(c)
                break
    mappp = {}
    idx =0
    for i in correct_clusters:
        if i not in mappp.keys():
            mappp[i] = idx
            idx += 1
    out = []
    for el in correct_clusters:
        out.append(mappp[el])
    return np.array(out)

In [629]:
correct_labels = get_correct_clusters(text)
correct_labels

array([ 0,  1,  2,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 11, 11, 11, 11,
       11, 11, 12, 12, 13, 14, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 22,
       23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 35, 36, 36, 37,
       38, 38, 38, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 41, 41, 41, 42,
       42, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 56,
       57, 58, 59,  2, 60, 61, 62, 63, 64, 63, 64, 65, 65, 65, 65, 65, 66,
       67,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3, 68, 69, 70, 70, 70,
       71, 72, 73, 74, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 89, 90, 91, 27, 27, 26, 26, 26, 26, 26, 92])

In [630]:
correct_clusters = clusters_from_labels(text, correct_labels)
for cluster in correct_clusters[:30]:
    for line in cluster:
        print(line)
    print("\n" + "#"*10 + "\n")

/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

##########

''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611

##########

''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
"SSONTEX" SP.ZO.O IMPORT-EXPORT 03-879 WARSZAWA UL PRZECLAWSKA 5 NIP:113-01-17-669

##########

''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4, 02-285 WARSZAWA.POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O."SP.K. UL,POGRANICZNA 2/4 02-285 WARSZAWA
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02--285 WARSZAWA
"TOPEX SP. Z O. O." SP. K. UL.POGRANICZNA 2/4,02-285 WARSZAWA,POLAND
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2

## Ocena jakości klasteryzacji

In [631]:
test_cosine = text_clusters(text,cosine_similarity,preprocess=False)
test_euclid = text_clusters(text,euclidean_distance,preprocess=False)
test_dice = text_clusters(text,dice,preprocess=False)
test_lcs = text_clusters(text,lcs,preprocess=False)
test_leven = text_clusters(text,levenshtein_distance,preprocess=False)

In [632]:
test_cosine_stoplist01 = text_clusters(text,cosine_similarity)
test_euclid_stoplist01 = text_clusters(text,euclidean_distance)
test_dice_stoplist01 =   text_clusters(text,dice)
test_lcs_stoplist01 =    text_clusters(text,lcs)
test_leven_stoplist01 =  text_clusters(text,levenshtein_distance)

In [633]:
test_cosine_stoplist03 = text_clusters(text,cosine_similarity,frequency=0.3)
test_euclid_stoplist03 = text_clusters(text,euclidean_distance,frequency=0.3)
test_dice_stoplist03 =   text_clusters(text,dice,frequency=0.3)
test_lcs_stoplist03 =    text_clusters(text,lcs,frequency=0.3)
test_leven_stoplist03 =  text_clusters(text,levenshtein_distance,frequency=0.3)

In [634]:
print("Cosine similarity:")
print("Davies bouldin index, no stoplist:      ", davies_bouldin_index(test_cosine, cosine_similarity))
print("Davies bouldin index, stoplist freq 0.1:", davies_bouldin_index(test_cosine_stoplist01, cosine_similarity))
print("Davies bouldin index, stoplist freq 0.3:", davies_bouldin_index(test_cosine_stoplist03, cosine_similarity))
print("Davies bouldin index, correct clusters: ", davies_bouldin_index(correct_clusters, cosine_similarity))
print("Dunn  index, no stoplist:               ", dunn_index(test_cosine, cosine_similarity))
print("Dunn  index, stoplist freq 0.1:         ", dunn_index(test_cosine_stoplist01, cosine_similarity))
print("Dunn  index, stoplist freq 0.3:         ", dunn_index(test_cosine_stoplist03, cosine_similarity))
print("Dunn  index, correct clusters:          ", dunn_index(test_cosine_stoplist03, cosine_similarity))

Cosine similarity:
Davies bouldin index, no stoplist:       0.4501895299204605
Davies bouldin index, stoplist freq 0.1: 0.6650400301775805
Davies bouldin index, stoplist freq 0.3: 0.6650400301775805
Davies bouldin index, correct clusters:  0.8782967078471439
Dunn  index, no stoplist:                0.5997414798288301
Dunn  index, stoplist freq 0.1:          0.05865554470251161
Dunn  index, stoplist freq 0.3:          0.05865554470251161
Dunn  index, correct clusters:           0.05865554470251161


In [635]:
print("Euclidean distance:")
print("Davies bouldin index, no stoplist:      ", davies_bouldin_index(test_euclid, euclidean_distance))
print("Davies bouldin index, stoplist freq 0.1:", davies_bouldin_index(test_euclid_stoplist01, euclidean_distance))
print("Davies bouldin index, stoplist freq 0.3:", davies_bouldin_index(test_euclid_stoplist03, euclidean_distance))
print("Davies bouldin index, correct clusters: ", davies_bouldin_index(correct_clusters, euclidean_distance))
print("Dunn  index, no stoplist:               ", dunn_index(test_euclid, euclidean_distance))
print("Dunn  index, stoplist freq 0.1:         ", dunn_index(test_euclid_stoplist01, euclidean_distance))
print("Dunn  index, stoplist freq 0.3:         ", dunn_index(test_euclid_stoplist03, euclidean_distance))
print("Dunn  index, correct clusters:          ", dunn_index(test_euclid_stoplist03, euclidean_distance))

Euclidean distance:
Davies bouldin index, no stoplist:       0.4574647387919868
Davies bouldin index, stoplist freq 0.1: 0.7283985264481668
Davies bouldin index, stoplist freq 0.3: 0.7283985264481668
Davies bouldin index, correct clusters:  0.8272660004274589
Dunn  index, no stoplist:                0.7710880102664802
Dunn  index, stoplist freq 0.1:          0.3541847573150378
Dunn  index, stoplist freq 0.3:          0.3541847573150378
Dunn  index, correct clusters:           0.3541847573150378


In [636]:
print("Dice index:")
print("Davies bouldin index, no stoplist:      ", davies_bouldin_index(test_dice, dice))
print("Davies bouldin index, stoplist freq 0.1:", davies_bouldin_index(test_dice_stoplist01, dice))
print("Davies bouldin index, stoplist freq 0.3:", davies_bouldin_index(test_dice_stoplist03, dice))
print("Davies bouldin index, correct clusters: ", davies_bouldin_index(correct_clusters, dice))
print("Dunn  index, no stoplist:               ", dunn_index(test_dice, dice))
print("Dunn  index, stoplist freq 0.1:         ", dunn_index(test_dice_stoplist01, dice))
print("Dunn  index, stoplist freq 0.3:         ", dunn_index(test_dice_stoplist03, dice))
print("Dunn  index, correct clusters:          ", dunn_index(test_dice_stoplist03, dice))

Dice index:
Davies bouldin index, no stoplist:       0.36151578099698206
Davies bouldin index, stoplist freq 0.1: 0.6890535811381948
Davies bouldin index, stoplist freq 0.3: 0.6890535811381948
Davies bouldin index, correct clusters:  0.8342088232852144
Dunn  index, no stoplist:                0.7047997112955611
Dunn  index, stoplist freq 0.1:          0.06987577639751555
Dunn  index, stoplist freq 0.3:          0.06987577639751555
Dunn  index, correct clusters:           0.06987577639751555


In [637]:
print("Longest common subsequence:")
print("Davies bouldin index, no stoplist:      ", davies_bouldin_index(test_lcs, lcs))
print("Davies bouldin index, stoplist freq 0.1:", davies_bouldin_index(test_lcs_stoplist01, lcs))
print("Davies bouldin index, stoplist freq 0.3:", davies_bouldin_index(test_lcs_stoplist03, lcs))
print("Davies bouldin index, correct clusters: ", davies_bouldin_index(correct_clusters, lcs))
print("Dunn  index, no stoplist:               ", dunn_index(test_lcs, lcs))
print("Dunn  index, stoplist freq 0.1:         ", dunn_index(test_lcs_stoplist01, lcs))
print("Dunn  index, stoplist freq 0.3:         ", dunn_index(test_lcs_stoplist03, lcs))
print("Dunn  index, correct clusters:          ", dunn_index(test_lcs_stoplist03, lcs))

Longest common subsequence:
Davies bouldin index, no stoplist:       0.5227146703887523
Davies bouldin index, stoplist freq 0.1: 0.6527440253349105
Davies bouldin index, stoplist freq 0.3: 0.6527440253349105
Davies bouldin index, correct clusters:  0.7805020830229776
Dunn  index, no stoplist:                0.46591889559965494
Dunn  index, stoplist freq 0.1:          0.0690045248868779
Dunn  index, stoplist freq 0.3:          0.0690045248868779
Dunn  index, correct clusters:           0.0690045248868779


In [638]:
print("Levenshtein distance:")
print("Davies bouldin index, no stoplist:      ", davies_bouldin_index(test_leven, levenshtein_distance))
print("Davies bouldin index, stoplist freq 0.1:", davies_bouldin_index(test_leven_stoplist01, levenshtein_distance))
print("Davies bouldin index, stoplist freq 0.3:", davies_bouldin_index(test_leven_stoplist03, levenshtein_distance))
print("Davies bouldin index, correct clusters: ", davies_bouldin_index(correct_clusters, levenshtein_distance))
print("Dunn  index, no stoplist:               ", dunn_index(test_leven, levenshtein_distance))
print("Dunn  index, stoplist freq 0.1:         ", dunn_index(test_leven_stoplist01, levenshtein_distance))
print("Dunn  index, stoplist freq 0.3:         ", dunn_index(test_leven_stoplist03, levenshtein_distance))
print("Dunn  index, correct clusters:          ", dunn_index(test_leven_stoplist03, levenshtein_distance))

Levenshtein distance:
Davies bouldin index, no stoplist:       0.4300949925867515
Davies bouldin index, stoplist freq 0.1: 0.5669856135969349
Davies bouldin index, stoplist freq 0.3: 0.5669856135969349
Davies bouldin index, correct clusters:  0.7195335656386141
Dunn  index, no stoplist:                0.500862812769629
Dunn  index, stoplist freq 0.1:          0.09783480352846832
Dunn  index, stoplist freq 0.3:          0.09783480352846832
Dunn  index, correct clusters:           0.09783480352846832


No i rzeczywiście wyniki wychodzą dosyć sensowne. Dane bez stoplisty są sklasteryzowne najgorzej, dane ze stoplistą lepiej, ale nigdy nie doganiają danych wzorcowych - a przynajmniej w przypadku indexu Davies Bouldina. W przypadku indeksu dunna to robią, ale nie jestem pewien czy tak powinno być. Może to być kwestia zastosowanej metryki w funkcji dunn_index (licznik i mianownik)

## Poprawa jakości klasteryzacji

6. Czy masz jakiś pomysł na poprawę jakości klasteryzacji w tym zadaniu?


* Lepszy preprocessing danych wejściowych - w tym zadaniu preprocessing nie został wykonany w najlepszy sposób, przez co niektóre wyrazy, które powinny występować oddzielnie zostały połączone

* Dobór parametrów modelu - zamiana parametrów takich jak eps może mieć wpływ na jakość klasteryzacji

* Wykorzystanie innych algorytmów - różne algorytmy klasteryzacji mają różne właściwości. W tym zadaniu użyto DBSCAN. Alternatywnimi algorytmami są np K-means, Gaussian Mixture Models

* Dobieranie odpowiednich metryk - metryka powinna być odpowiednio dostosowana do danych wejściowych

* Użycie sieci neuronowej. Można wytrenować odpowiednio mądry model który będzie sam potrafił zrobić taki clustering bez potrzeby używania algorytmów